In [1]:
import sys  
sys.path.insert(0, '/scratch/szym/introns/noncanonical_introns')

from representation_wrapper import RepresentationWrapper
from Bio.SeqIO.FastaIO import SimpleFastaParser
import time
import pickle
import pandas

In [10]:
started = time.time()
file = '/scratch/szym/introns/selected_introns.fasta'
with open(file, "r") as handle:
    sequences = list(SimpleFastaParser(handle))
    
types = [0]*len(sequences)
for i, s in enumerate(sequences):
    class_signature=s[0][-2:]
    if class_signature == "KX":    
        # 0  for conventional intron
        types[i] = (0)
    else:
        # 1 for nonconventional
        types[i] = (1)

print("Elapsed",time.time()-started, "seconds")

Elapsed 5.875684022903442 seconds


In [11]:
started = time.time()
representations = RepresentationWrapper(sequences, "list_of_FASTA_pairs", "ACTG", 'N', types)
print("Elapsed",time.time()-started, "seconds")

Elapsed 0.8660333156585693 seconds


In [5]:
started = time.time()
x=representations.to_pandas_dataframe()
print("Elapsed",time.time()-started, "seconds")

Elapsed 47.88313603401184 seconds


In [12]:
representations.sanity_check()

(True, 'Everything OK')

In [10]:
x

<10000x256 sparse matrix of type '<class 'numpy.float64'>'
	with 1939343 stored elements in Compressed Sparse Row format>

In [13]:
started = time.time()
x=representations.to_TfIdf(space_treatment = "exclude")
print("Elapsed",time.time()-started, "seconds")

Elapsed 874.4904716014862 seconds


In [ ]:
started = time.time()
x=representations.to_TfIdf(space_treatment = "include")
print("Elapsed",time.time()-started, "seconds")

In [ ]:
started = time.time()
x=representations.to_bag_of_words(space_treatment = "exclude")
print("Elapsed",time.time()-started, "seconds")

In [ ]:
started = time.time()
x=representations.to_bag_of_words(space_treatment = "include")
print("Elapsed",time.time()-started, "seconds")

In [8]:
col_count = len(representations.to_pandas_dataframe().columns)
representations.to_pandas_dataframe().columns = ["c%d"%i for i in range(col_count)]
started = time.time()
representations.to_pandas_dataframe().to_parquet("/scratch/szym/subset.parquet")
print("Elapsed",time.time()-started, "seconds")

TypeError: Sparse pandas data (column c0) not supported.

In [1]:
###VISUALISATION OF LIBRARY WORKING

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
corpus = ['AAAAAAAAAAAAAA', 'BBBBBBBBBBBBBBBBBBBBBBBB',
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
]
vectorizer = CountVectorizer(analyzer = "word", token_pattern="(?=([%s]{%d}))"%("document", 4), lowercase=True)
X = vectorizer.fit_transform(corpus )
print(X)
print(vectorizer.get_feature_names())
for s in range(4):
    for ind, word in enumerate(vectorizer.get_feature_names()):
         if X[s, ind] != 0:
             print(s, word, X[s, ind])

  (2, 2)	1
  (2, 5)	1
  (2, 1)	1
  (2, 6)	1
  (2, 4)	1
  (3, 2)	2
  (3, 5)	2
  (3, 1)	2
  (3, 6)	2
  (3, 4)	2
  (3, 3)	1
  (3, 0)	1
  (5, 2)	1
  (5, 5)	1
  (5, 1)	1
  (5, 6)	1
  (5, 4)	1
['cond', 'cume', 'docu', 'econ', 'ment', 'ocum', 'umen']
2 cume 1
2 docu 1
2 ment 1
2 ocum 1
2 umen 1
3 cond 1
3 cume 2
3 docu 2
3 econ 1
3 ment 2
3 ocum 2
3 umen 2
